In [2]:
import pandas as pd

In [3]:
#df = pd.read_csv("vaers_cleaned.csv")
df = pd.read_csv("dataset_with_entities (2).csv")

In [4]:
def stratified_sample(df, frac=None, n=None, random_state=42):
    """
    Stratified sampling by AGE_GROUP, SEVERE, and VAX_NAME.
    You can specify either `frac` (fraction of each group) or `n` (fixed number per group).
    
    Args:
        df (pd.DataFrame): Input dataframe with columns AGE_YRS, HOSPITAL, L_THREAT, DIED, VAX_NAME
        frac (float, optional): Fraction of rows to sample from each group.
        n (int, optional): Fixed number of rows to sample from each group.
        random_state (int): Random seed for reproducibility.
        
    Returns:
        pd.DataFrame: Stratified sample
    """
    import pandas as pd
    
    # Age bins
    bins = [-1, 17, 30, 45, 60, 75, 120]
    labels = ["child", "young_adult", "adult", "middle_age", "senior", "elderly"]
    df = df.copy()
    df["AGE_GROUP"] = pd.cut(df["AGE_YRS"], bins=bins, labels=labels)
    
    # Severity flag (hospital + death + life threat)
    df["SEVERE"] = df[["HOSPITAL", "L_THREAT", "DIED"]].apply(
        lambda x: 1 if ("Y" in str(x.values)) else 0, axis=1
    )
    
    # Grouping keys
    group_keys = ["AGE_GROUP", "SEVERE", "VAX_NAME"]
    
    # Apply stratified sampling
    if frac is not None:
        stratified = (
            df.groupby(group_keys, group_keys=False)
              .apply(lambda x: x.sample(frac=frac, random_state=random_state)
                     if len(x) > 0 else x)
        )
    elif n is not None:
        stratified = (
            df.groupby(group_keys, group_keys=False)
              .apply(lambda x: x.sample(n=min(len(x), n), random_state=random_state)
                     if len(x) > 0 else x)
        )
    else:
        raise ValueError("You must specify either `frac` or `n` for sampling.")
    
    return stratified


In [7]:
gold_subset = stratified_sample(df, frac=0.02)
#sample = stratified_sample(df, n=100)


C:\Users\sugan\AppData\Local\Temp\ipykernel_9888\1906160151.py:34: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(group_keys, group_keys=False)
C:\Users\sugan\AppData\Local\Temp\ipykernel_9888\1906160151.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(frac=frac, random_state=random_state)


In [8]:
print(f"✅ Gold Data subset shape: {gold_subset.shape}")
#print(gold_subset.head(3)[["SYMPTOM_TEXT", "SYMPTOMS_STRUCT","AGE_GROUP", "SEVERE"]])

✅ Gold Data subset shape: (361, 18)


In [6]:
#gold_subset['SYMPTOM_TEXT'].apply(type).value_counts()


SYMPTOM_TEXT
<class 'str'>      18397
<class 'float'>       25
Name: count, dtype: int64

In [9]:
# Save for annotation
gold_subset.to_csv("./sample1.csv", index=False)
print("💾 Saved gold subset for manual annotation -> ade_gold_subset_20k.csv")

💾 Saved gold subset for manual annotation -> ade_gold_subset_20k.csv


In [10]:
all_symptoms = gold_subset['SYMPTOMS_STRUCT'].tail(50).tolist()

In [11]:
all_symptoms

['Acute respiratory distress syndrome | COVID-19 | Dyspnoea | Fatigue | SARS-CoV-2 test positive',
 'Chest X-ray | Renal impairment | Cough | Fall | Muscular weakness | Pneumonia',
 'Malignant neoplasm of ampulla of Vater |  |  |  | ',
 'Acute respiratory failure | Computerised tomogram thorax abnormal | Inappropriate schedule of product administration | SARS-CoV-2 test positive | Anticoagulant therapy | Condition aggravated | Oropharyngeal pain | Scan with contrast abnormal | Bronchiectasis | Cough | Pain | Tachypnoea | COVID-19 | Death | Pulmonary thrombosis | COVID-19 pneumonia | Headache | Respiratory disorder',
 'COVID-19 | SARS-CoV-2 test positive | Cough | Malaise | Pain | Rhinorrhoea',
 'COVID-19 | SARS-CoV-2 test positive | Vaccine breakthrough infection |  | ',
 "Acute kidney injury | COVID-19 | Drug screen negative | Hyperkalaemia | Mental status changes | SARS-CoV-2 test positive | Vertebral foraminal stenosis | Alcohol test negative | Computerised tomogram spine | Dysphagi